## Using the provided listenbrainz_model.py file

In [6]:
import os
import pandas as pd
import numpy as np
import json

# Import our model functions from the provided module.
# Make sure that the project root is in PYTHONPATH.
import listenbrainz_model as lb

# Define the working directory (adjust if needed)
working_root = "/mnt/j/MusicBrainz/working"

# Step 1: Load the data matrix
data_matrix_path = os.path.join(working_root, "userid-artist-counts.csv")
matrix_artists, matrix_users, plays = lb.load_data_matrix(data_matrix_path)
print(f"Data matrix shape: {plays.shape}")

# Step 2: Build the ALS model
model = lb.build_model(plays)
print("Model training complete!")

# Step 3: Load the artist mapping.
# Option 1: Use our JSON file created earlier
artist_map_path = os.path.join(working_root, "artist_mapping.json")
with open(artist_map_path, "r", encoding="utf-8") as f:
    artist_mapping = json.load(f)

# Option 2: Use the provided get_artist_map function if using the MusicBrainz CSV directly:
# musicbrainz_artist_csv = "/mnt/j/MusicBrainz/musicbrainz_artist.csv"
# artist_mapping = lb.get_artist_map(musicbrainz_artist_csv)

# Step 4: Query the model for similar artists.
# For demonstration, we select an artist from the matrix.
# We'll choose the first artist from matrix_artists (you can choose a different one if desired)
target_artist_mbid = matrix_artists[0]
print(f"Using target artist MBID: {target_artist_mbid}")

# Find the index of the target artist
try:
    target_index = lb.artist_index(matrix_artists, target_artist_mbid)
except ValueError as e:
    print(f"Error: {e}")
    target_index = None

if target_index is not None:
    # Retrieve similar items from the model
    similar_ids, scores = model.similar_items(target_index, N=20)
    
    # Prepare a DataFrame with MBIDs, names, and similarity scores
    similar_artists = []
    for idx, score in zip(similar_ids, scores):
        mbid = matrix_artists[idx]
        name = artist_mapping.get(mbid, "unknown")
        similar_artists.append((mbid, name, score))
    
    df_similar = pd.DataFrame(similar_artists, columns=["Artist MBID", "Artist Name", "Score"])
    print(df_similar)


Data matrix shape: (9087, 163209)


  0%|          | 0/15 [00:00<?, ?it/s]

Model training complete!
Using target artist MBID: 00006766-a163-44eb-b6f1-1d82973b95ec
                             Artist MBID                 Artist Name     Score
0   00006766-a163-44eb-b6f1-1d82973b95ec                       Heron  1.000000
1   488bc172-1e96-43ac-874f-2aa6c68a8d5f        The Depth Beneath Us  0.977081
2   2255a6ed-3149-4f1c-a127-ccf6f320e63b            Deer Park Ranger  0.975456
3   7dec8f78-33d5-42d4-95ca-3a1b314b591c            A River Crossing  0.975456
4   28d18fb2-80d1-4ebc-8b2a-491458b35161                       Waves  0.975455
5   bcc19451-48f0-4753-b2ad-f87720ea8570  Old Seas / Young Mountains  0.975455
6   44f979db-4415-46c2-94ee-217427cf5e36                     Soonago  0.975455
7   f6819f90-9a94-4cf2-a8a5-638135b23780                  Satellites  0.975455
8   38a689b1-31ba-4d2e-b314-c62f7a7f4f85            Six Days of Calm  0.975455
9   d3b97060-f1c6-4d92-a1e9-46f1b8a50a80            Catch The Breeze  0.975455
10  defbcce5-e467-4d4e-99f0-91624086b340   

## Model Building with `listenbrainz_model.py`

We leveraged the provided `listenbrainz_model.py` module to build and query our collaborative filtering model. The key steps in this process were as follows:

1. **Loading the Data Matrix**
   - **Function Used:** `load_data_matrix(user_artist_counts_path)`
   - **Process:**  
     - The aggregated CSV file (`userid-artist-counts.csv`), which contains user IDs, artist MBIDs, and play counts, is loaded using Pandas.
     - The `user` and `artist` columns are treated as categorical data, and their codes are used to construct a sparse matrix (using SciPy's `coo_matrix`), representing the number of listens per user-artist pair.
   - **Output:**  
     - `matrix_artists`: An array (Pandas Categorical) containing artist MBIDs.
     - `matrix_users`: An array containing user IDs.
     - `plays`: A sparse matrix of shape *(num_users, num_artists)* holding the listen counts.

2. **Training the ALS Model**
   - **Function Used:** `build_model(plays)`
   - **Process:**  
     - BM25 weighting is applied to the sparse matrix to adjust the raw play counts, reducing the influence of extremely popular items and over-active users.
     - The Alternating Least Squares (ALS) model from the Implicit library is then initialized with parameters (e.g., 64 latent factors, regularization of 0.05, and an alpha of 2.0) and trained on the weighted matrix.
   - **Output:**  
     A trained ALS model that can be used to compute similarities between items (artists).

3. **Loading the Artist Mapping**
   - **Function Used:** `get_artist_map(musicbrainz_artist_path)`  
     *(Alternatively, a JSON mapping file generated from the `musicbrainz_artist.csv` file is used.)*
   - **Process:**  
     - The mapping file provides a dictionary that links each artist MBID to its textual name.
     - This mapping is critical for converting the model’s internal recommendations (which use MBIDs) into human-readable artist names.
   - **Output:**  
     A dictionary mapping artist MBIDs to their names.

4. **Querying the Model for Similar Artists**
   - **Function Used:** `artist_index(artists, artist_mbid)` along with the model's `similar_items` method.
   - **Process:**  
     - First, we identify the index of a target artist in the `matrix_artists` array using `artist_index()`.
     - Next, we call the model's `similar_items` method with the target artist’s index to obtain a list of similar artist indices and their corresponding similarity scores.
     - Finally, we convert these indices back to artist MBIDs and use the artist mapping to display the artist names.
   - **Output:**  
     A list (displayed as a Pandas DataFrame) of similar artists, showing each artist's MBID, name, and similarity score.

This structured process—from data matrix creation to model training and querying—forms the backbone of our collaborative filtering system. It allows us to generate artist recommendations based on implicit listening behavior, while also providing human-readable outputs for evaluation and analysis.


## Real-World Examples: Finding Similar Artists

Below is a code snippet that queries our trained collaborative filtering model for similar artists to two well-known bands:
- **The Beatles:** `b10bbbfc-cf9e-42e0-be17-e2c3e1d2600d`
- **Soda Stereo:** `3f8a5e5b-c24b-4068-9f1c-afad8829e06b`

The code uses the `artist_index` function from `listenbrainz_model.py` to find the index of the target artist in our data matrix, then calls the model’s `similar_items` method to retrieve the top 20 similar artists. Finally, it maps these artist MBIDs to human-readable names using our artist mapping.


In [7]:
import pandas as pd
import numpy as np

# Function to retrieve similar artists for a given target artist MBID
def get_similar_artists(target_artist_mbid, matrix_artists, model, artist_mapping, N=20):
    try:
        # Get the index of the target artist from the matrix
        target_idx = lb.artist_index(matrix_artists, target_artist_mbid)
    except ValueError as e:
        print(f"Error: {e}")
        return None

    # Retrieve similar artist indices and their similarity scores
    similar_ids, scores = model.similar_items(target_idx, N=N)
    
    # Map the indices back to artist MBIDs and then to names using the artist mapping
    similar_artists = [
        (matrix_artists[i], artist_mapping.get(matrix_artists[i], "unknown"), score)
        for i, score in zip(similar_ids, scores)
    ]
    
    return pd.DataFrame(similar_artists, columns=["Artist MBID", "Artist Name", "Score"])

# Define our target artists with their MusicBrainz IDs
target_artists = {
    "The Beatles": "b10bbbfc-cf9e-42e0-be17-e2c3e1d2600d",
    "Soda Stereo": "3f8a5e5b-c24b-4068-9f1c-afad8829e06b"
}

# Loop over each target artist and display their similar artists
for artist_name, mbid in target_artists.items():
    print(f"Similar artists for {artist_name} (MBID: {mbid}):")
    df_similar = get_similar_artists(mbid, matrix_artists, model, artist_mapping, N=20)
    if df_similar is not None:
        display(df_similar)
    print("\n")

Similar artists for The Beatles (MBID: b10bbbfc-cf9e-42e0-be17-e2c3e1d2600d):


,Artist MBID,Artist Name,Score
0,b10bbbfc-cf9e-42e0-be17-e2c3e1d2600d,The Beatles,1.000000
1,83d91898-7763-47d7-b03b-b92132375c47,Pink Floyd,0.822775
2,b071f9fa-14b0-4217-8e97-eb41da73f598,The Rolling Stones,0.780081
3,0383dadf-2a4e-4d10-a46a-e9e041da8eb3,Queen,0.752612
4,72c536dc-7137-4477-a521-567eeb840fa8,Bob Dylan,0.751956
5,9fdaa16b-a6c4-4831-b87c-bc9ca8ce7eaa,The Who,0.736290
6,9e0e2b01-41db-4008-bd8b-988977d6019a,The Police,0.727633
7,678d88b2-87b0-403b-b63d-5da7465aecc3,Led Zeppelin,0.723639
8,ba550d0e-adac-4864-b88b-407cab5e76af,Paul McCartney,0.720423
9,39ab1aed-75e0-4140-bd47-540276886b60,Oasis,0.720175




Similar artists for Soda Stereo (MBID: 3f8a5e5b-c24b-4068-9f1c-afad8829e06b):


,Artist MBID,Artist Name,Score
0,3f8a5e5b-c24b-4068-9f1c-afad8829e06b,Soda Stereo,1.000000
1,77814bfb-f1b0-47fd-8492-65d6f72c246d,Caifanes,0.840883
2,e6e1e76f-afee-4990-aad0-056199d94918,Babasónicos,0.811058
3,dd999ced-1dd4-44b8-ab67-ee5ec23e52b7,Charly García,0.803720
4,985b05cf-1a57-40dc-8ee7-6596440a6821,Virus,0.802414
5,54bd120a-6bac-4d7c-8b10-ba167bc7e26c,Fito Páez,0.800385
6,0bca46d2-184f-41d0-9abe-37c99d3c023f,Los Tres,0.798711
7,acc7cb5a-8f3a-4f61-b199-a62d45374dec,Enjambre,0.789728
8,1348c8de-92ce-4f4f-9233-728958f35259,Cabezones,0.782801
9,979d01bc-a4da-4c80-8cc1-be40bd35cd5e,Gustavo Cerati,0.781372
